This file has code that:
    (1) follows segmentation performed in old pipeline exactly (so that results could be replicated)
    (2) saves 3D scan for analysis offline
    (3) adapts old segmentation to newer CAIMAN pipeline for comparison between the two methods

In [125]:
from pipeline.utils import caiman_interface as cmn
import json
import uuid
import os
from pipeline import experiment, meso
from pipeline.utils import performance
import scanreader
import numpy as np

In [39]:
key = {'animal_id': 30415, 'session': 5, 'scan_idx': 10, 'field': 3, 'channel': 1}

In [40]:
field_id = key['field'] - 1
channel = key['channel'] - 1
image_height, image_width = (meso.ScanInfo.Field() & key).fetch1('px_height', 'px_width')
num_frames = (meso.ScanInfo() & key).fetch1('nframes')

In [126]:
key2 = key = {'animal_id': 30415, 'session': 5, 'scan_idx': 10}

In [132]:
meso.ScanInfo.Field() & key

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,pipe_version,field,px_height height in pixels,px_width width in pixels,um_height height in microns,um_width width in microns,x (um) center of field in the motor coordinate system,y (um) center of field in the motor coordinate system,z (um) absolute depth with respect to the surface of the cortex,delay_image (ms) delay between the start of the scan and pixels in this field,roi ROI to which this field belongs
30415,5,10,1,1,240,240,600.0,600.0,-1250.0,54.0,171.0,=BLOB=,0
30415,5,10,1,2,240,240,600.0,600.0,-1250.0,54.0,176.0,=BLOB=,0
30415,5,10,1,3,240,240,600.0,600.0,-1250.0,54.0,181.0,=BLOB=,0
30415,5,10,1,4,240,240,600.0,600.0,-1250.0,54.0,186.0,=BLOB=,0
30415,5,10,1,5,240,240,600.0,600.0,-1250.0,54.0,191.0,=BLOB=,0
30415,5,10,1,6,240,240,600.0,600.0,-1250.0,54.0,196.0,=BLOB=,0
30415,5,10,1,7,240,240,600.0,600.0,-1250.0,54.0,201.0,=BLOB=,0
30415,5,10,1,8,240,240,600.0,600.0,-1250.0,54.0,206.0,=BLOB=,0
30415,5,10,1,9,240,240,600.0,600.0,-1250.0,54.0,211.0,=BLOB=,0
30415,5,10,1,10,240,240,600.0,600.0,-1250.0,54.0,216.0,=BLOB=,0


In [136]:
channel = 0
image_height, image_width = (meso.ScanInfo.Field() & key).fetch('px_height', 'px_width')
image_height, image_width = image_height[0], image_width[0]
num_frames = (meso.ScanInfo() & key).fetch('nframes')[0]

In [138]:
num_frames

61717

In [139]:
scan_filename = (experiment.Scan() & key).local_filenames_as_wildcard
scan = scanreader.read_scan(scan_filename)

In [141]:
fields = scan[:,:,:,channel,:]
field = fields - np.min(fields)
np.save('tmp/3dscan.npy', fields)

In [108]:
filename = "/tmp/caiman-{}_d1_{}_d2_{}_d3_{}_order_C_frames_{}_.mmap".format(
                uuid.uuid4(), image_height, image_width, num_frames, num_frames
            )
mmap_shape = (image_height * image_width, num_frames)
mmap_scan = np.memmap(filename, mode='w+', shape=mmap_shape, dtype=np.float32)

In [109]:
# Map: Correct scan and save in memmap scan
f = performance.parallel_save_memmap  # function to map
raster_phase = (meso.RasterCorrection() & key).fetch1("raster_phase")
fill_fraction = (meso.ScanInfo() & key).fetch1("fill_fraction")
y_shifts, x_shifts = (meso.MotionCorrection() & key).fetch1(
    "y_shifts", "x_shifts"
)

In [110]:
kwargs = {
    "raster_phase": raster_phase,
    "fill_fraction": fill_fraction,
    "y_shifts": y_shifts,
    "x_shifts": x_shifts,
    "mmap_scan": mmap_scan,
}
results = performance.map_frames(
    f, scan, field_id=field_id, channel=channel, kwargs=kwargs
)

In [111]:
# Reduce: Use the minimum values to make memory mapped scan nonnegative
mmap_scan -= np.min(results)  # bit inefficient but necessary

In [79]:
# Set CNMF parameters
## Set general parameters
kwargs = {}
kwargs["num_background_components"] = 1
kwargs["merge_threshold"] = 0.7
kwargs["fps"] = (meso.ScanInfo() & key).fetch1("fps")

# Set params specific to method and segmentation target
target = (meso.SegmentationTask() & key).fetch1("compartment")
seg_method = (meso.SegmentationTask() & key).fetch1("segmentation_method")
if seg_method == 2:  # nmf
    if target == "axon":
        kwargs["init_on_patches"] = True
        kwargs["proportion_patch_overlap"] = 0.2  # 20% overlap
        kwargs["num_components_per_patch"] = 15
        kwargs["init_method"] = "sparse_nmf"
        kwargs["snmf_alpha"] = 500  # 10^2 to 10^3.5 is a good range
        kwargs["patch_size"] = tuple(
            50 / (meso.ScanInfo.Field() & key).microns_per_pixel
        )  # 50 x 50 microns
    elif target == "bouton":
        kwargs["init_on_patches"] = False
        kwargs["num_components"] = (
            meso.SegmentationTask() & key
        ).estimate_num_components()
        kwargs["init_method"] = "greedy_roi"
        kwargs["soma_diameter"] = tuple(
            2 / (meso.ScanInfo.Field() & key).microns_per_pixel
        )
    else:  # soma
        kwargs["init_on_patches"] = False
        kwargs["num_components"] = (
            meso.SegmentationTask() & key
        ).estimate_num_components()
        kwargs["init_method"] = "greedy_roi"
        kwargs["soma_diameter"] = tuple(
            14 / (meso.ScanInfo.Field() & key).microns_per_pixel
        )
else:  # nmf-new
    kwargs["init_on_patches"] = True
    kwargs["proportion_patch_overlap"] = 0.2  # 20% overlap
    if target == "axon":
        kwargs["num_components_per_patch"] = 15
        kwargs["init_method"] = "sparse_nmf"
        kwargs["snmf_alpha"] = 500  # 10^2 to 10^3.5 is a good range
        kwargs["patch_size"] = tuple(
            50 / (meso.ScanInfo.Field() & key).microns_per_pixel
        )  # 50 x 50 microns
    elif target == "bouton":
        kwargs["num_components_per_patch"] = 5
        kwargs["init_method"] = "greedy_roi"
        kwargs["patch_size"] = tuple(
            20 / (meso.ScanInfo.Field() & key).microns_per_pixel
        )  # 20 x 20 microns
        kwargs["soma_diameter"] = tuple(
            2 / (meso.ScanInfo.Field() & key).microns_per_pixel
        )
    else:  # soma
        kwargs["num_components_per_patch"] = 6
        kwargs["init_method"] = "greedy_roi"
        kwargs["patch_size"] = tuple(
            50 / (meso.ScanInfo.Field() & key).microns_per_pixel
        )
        kwargs["soma_diameter"] = tuple(
            8 / (meso.ScanInfo.Field() & key).microns_per_pixel
        )

## Set performance/execution parameters (heuristically), decrease if memory overflows
kwargs["num_processes"] = 8  # Set to None for all cores available
kwargs["num_pixels_per_process"] = 10000


In [80]:
kwargs

{'num_background_components': 1,
 'merge_threshold': 0.7,
 'fps': 8.3091,
 'init_on_patches': True,
 'proportion_patch_overlap': 0.2,
 'num_components_per_patch': 6,
 'init_method': 'greedy_roi',
 'patch_size': (20.0, 20.0),
 'soma_diameter': (3.2, 3.2),
 'num_processes': 8,
 'num_pixels_per_process': 10000}

In [113]:
scan_ = mmap_scan.reshape(
    (image_height, image_width, num_frames), order="F"
)


In [84]:
cnmf_result = cmn.extract_masks(scan_, mmap_scan, **kwargs)
(
    masks,
    traces,
    background_masks,
    background_traces,
    raw_traces,
) = cnmf_result


/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpre

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[[slice(*a) for a in ijSig]] -= dataSig.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidim

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rho[[slice(*a) for a in Mod]] -= temp.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is de

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the f

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[[slice(*a) for a in ijSig]] -= dataSig.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:543: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` i

...

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rho[[slice(*a) for a in Mod]] -= temp.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is de

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rho[[slice(*a) for a in Mod]] -= temp.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is de

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecate

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[[slice(*a) for a in ijSig]] -= dataSig.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:543: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp[[slice(*a) for a in Lag]] = coef
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated;

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[[slice(*a) for a in ijSig]] -= dataSig.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:543: FutureWarning: Using a non-tuple sequence for multidim

In [85]:
masks.shape

(240, 240, 682)

In [87]:
from caiman_interface import extract_masks

cnmf_result2 = extract_masks(scan_, mmap_scan, **kwargs)
(
    masks2,
    traces2,
    background_masks2,
    background_traces2,
    raw_traces2,
) = cnmf_result2

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpre

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:543: FutureWarning: Using a non-tuple sequence for multidimensional index

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rho[[slice(*a) for a in Mod]] -= temp.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is de

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpre

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v[[slice(*a) for a in Mod]] = np.amax(rho_filt, axis=-1)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for

...

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v[[slice(*a) for a in Mod]] = np.amax(rho_filt, axis=-1)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rho[[slice(*a) for a in Mod]] -= temp.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is de

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v[[slice(*a) for a in Mod]] = np.amax(rho_filt, axis=-1)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rho[[slice(*a) for a in Mod]] -= temp.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is de

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rho[[slice(*a) for a in Mod]] -= temp.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is de

In [89]:
masks2.shape

(240, 240, 682)

In [90]:
from caiman import load_memmap

In [91]:
filename = 'data/caiman/caiman_d1_240_d2_240_d3_1_order_C_frames_61717_.mmap'
mmap_scan2, (image_height2, image_width2), num_frames2 = load_memmap(filename)

In [92]:
scan_2 = mmap_scan2.reshape(
    (image_height2, image_width2, num_frames2), order="F"
)

In [93]:
cnmf_result3 = extract_masks(scan_2, mmap_scan2, **kwargs)
(
    masks3,
    traces3,
    background_masks3,
    background_traces3,
    raw_traces3,
) = cnmf_result3

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpre

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-t

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for mu

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpre

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[[slice(*a) for a in ijSig]] -= dataSig.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:543: FutureWarning: Using a non-tuple sequence for multidim

...

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecate

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecate

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v[[slice(*a) for a in Mod]] = np.amax(rho_filt, axis=-1)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional in

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[[slice(*a) for a in ijSig]] -= dataSig.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:543: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp[[slice(*a) for a in Lag]] = coef
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated;

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rho[[slice(*a) for a in Mod]] -= temp.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is de

In [97]:
masks3.shape

(240, 240, 747)

In [98]:
mmap_scan3 = mmap_scan2 - np.min(results)  # bit inefficient but necessary
scan_2 = mmap_scan3.reshape(
    (image_height2, image_width2, num_frames2), order="F"
)

In [ ]:
cnmf_result4 = extract_masks(scan_2, mmap_scan3, **kwargs)
(
    masks4,
    traces4,
    background_masks4,
    background_traces4,
    raw_traces4,
) = cnmf_result4
masks4.shape

In [102]:
os.remove(mmap_scan.filename)

In [ ]:
# remove mmap_scan using above cell and then reload and don't perform minimization and see if I replicate the ~750 masks in my segmentation

AttributeError: 'numpy.ndarray' object has no attribute 'filename'

In [100]:
cnmf_result4 = cmn.extract_masks(scan_2, mmap_scan2, **kwargs)
(
    masks4,
    traces4,
    background_masks4,
    background_traces4,
    raw_traces4,
) = cnmf_result4

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpre

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:543: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp[[slice(*a) for a in Lag]] = coef
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[[slice(*a) for a in ijSig]] -= dataSig.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:546: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated;

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[[slice(*a) for a in ijSig]] -= dataSig.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidim

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpre

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v[[slice(*a) for a in Mod]] = np.amax(rho_filt, axis=-1)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for

...

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dataTemp = np.array(Y[[slice(*a) for a in ijSig]].copy(), dtype=np.float)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:536: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[[slice(*a) for a in ijSig]] -= dataSig.copy()
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:543: FutureWarning: Using a non-tuple sequence for multidim

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v[[slice(*a) for a in Mod]] = np.amax(rho_filt, axis=-1)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional in

/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the f

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v[[slice(*a) for a in Mod]] = np.amax(rho_filt, axis=-1)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:524: FutureWarning: Using a non-tuple sequence for multidimensional in

/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:549: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rolling_filter, 1., rho[[slice(*a) for a in Mod]]**2)
/usr/local/lib/python3.6/dist-packages/caiman/source_extraction/cnmf/initialization.py:550: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v[[slice(*a) for a in Mod]] = np.amax(rho_filt, axis=-1)


In [101]:
masks4.shape

(240, 240, 748)

In [ ]:
masks4

In [107]:
np.save('masks_old4.npy', masks2)

In [1]:
 # save mmap_scan with baseline removed
# run new segmentation on mmap_scan
from segmentation_pipelines import save_as_memmap, extract_masks_adapted
from caiman import load_memmap
import numpy as np

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2024-02-16 20:16:01.590094: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 20:16:01.590246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 20:16:01.592699: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

In [3]:
# Save as memory mapped file in F order (that's how caiman wants it)
# mmap_filename = save_as_memmap(scan_, base_name='tmp/caiman').filename
mmap_filename = r'tmp/caiman_d1_240_d2_240_d3_1_order_C_frames_61717_.mmap'
# 'Load' scan
mmap_scan, (image_height, image_width), num_frames = load_memmap(mmap_filename)
mmap_scan

memmap([[ 470.29492,  470.29492,  470.29492, ...,  470.29492,  470.29492,
          470.29492],
        [ 648.11554,  785.96826,  694.48517, ...,  470.29492,  470.29492,
          470.29492],
        [ 731.94116, 1008.27826,  815.9369 , ...,  470.29492,  470.29492,
          470.29492],
        ...,
        [ 470.29492,  470.29492,  470.29492, ...,  920.5055 ,  731.3817 ,
          636.6928 ],
        [ 470.29492,  470.29492,  470.29492, ..., 1191.3706 ,  890.77185,
          918.3744 ],
        [ 470.29492,  470.29492,  470.29492, ...,  646.54565,  470.29492,
          470.29492]], dtype=float32)

In [4]:
kwargs = {'num_background_components': 1,
 'merge_threshold': 0.7,
 'fps': 8.3091,
 'init_on_patches': True,
 'proportion_patch_overlap': 0.2,
 'num_components_per_patch': 6,
 'init_method': 'greedy_roi',
 'patch_size': (20.0, 20.0),
 'soma_diameter': (3.2, 3.2),
 'num_processes': 8,
 'num_pixels_per_process': 10000}

In [5]:
scan_ = mmap_scan.reshape(
    (image_height, image_width, num_frames), order="F"
)


In [6]:
cnmf_result = extract_masks_adapted(scan_, mmap_scan, **kwargs)
(
    masks,
    traces,
    background_masks,
    background_traces,
    raw_traces,
) = cnmf_result

[Fri Feb 16 20:17:01 2024] Starting 8 processes...
[Fri Feb 16 20:17:02 2024] Initializing components...
[Fri Feb 16 20:27:17 2024] 934 components found...
[Fri Feb 16 20:27:17 2024] Removing bad components...


[Fri Feb 16 20:27:37 2024] 921 components remaining...
[Fri Feb 16 20:27:37 2024] Calculating noise per pixel...
[Fri Feb 16 20:27:39 2024] Updating masks...
[Fri Feb 16 20:29:00 2024] Updating traces...
[Fri Feb 16 20:29:39 2024] Merging overlapping (and temporally correlated) masks...
[Fri Feb 16 20:30:42 2024] Refining masks...
[Fri Feb 16 20:32:07 2024] Refining traces...
[Fri Feb 16 20:32:46 2024] Removing bad components...
[Fri Feb 16 20:33:29 2024] 642 components remaining...
[Fri Feb 16 20:33:29 2024] Done.


In [7]:
masks.shape

(240, 240, 642)

In [9]:
np.save('masks_new4.npy', masks)